In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale

In [2]:
df = pd.read_csv('Fully_Joined_OCI_Traffic_Table.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.columns

Index(['OBJECTID_12', 'Join_Count', 'TARGET_FID', 'OBJECTID_1', 'OBJECTID',
       'CITY_ID', 'KIVAPIN', 'LAYER', 'L_F_ADD', 'L_T_ADD', 'R_F_ADD',
       'R_T_ADD', 'PREFIX', 'STREETNAME', 'STREETTYPE', 'SUFFIX', 'LZIP',
       'RZIP', 'L_PLA_CODE', 'R_PLA_CODE', 'ONEWAYCODE', 'SPEED_LIMI',
       'MajorStree', 'ParkMaint', 'StreetPres', 'Shape_Leng', 'Status',
       'Maintenanc', 'Maintena_1', 'zID', 'zPaveClass', 'zLegFunCla',
       'zStreet', 'zFrom', 'zTo', 'zLength', 'zWidth', 'zArea', 'zEstOCI',
       'zInspectID', 'zInspectBy', 'zInspectDa', 'zInspectOC', 'zDate',
       'BUFF_DIST', 'ORIG_FID', 'link_id', 'volume', 'geometry',
       'Shape_Leng.1', 'Field1', 'link_id.1', 'volume_x', 'geometry.1',
       'link_id_plus', 'link_id_minus', 'volume_y', 'link_id_plus_exists',
       'link_id_minus_exists', 'link_id_is_even', 'mean_traffic_volume',
       'standard_deviation_traffic_volume', 'drop_this_observation',
       'ORIG_FID.1', 'Shape_Length'],
      dtype='object')

In [4]:
df = df[['OBJECTID_12', 'OBJECTID_1', 'OBJECTID', 'CITY_ID', 
      'zEstOCI', 'zInspectOC', 'link_id', 'mean_traffic_volume', 
        'standard_deviation_traffic_volume']]

In [5]:
df['scaled_traffic_volume'] = ""

In [6]:
#df = df.fillna(-1)

In [7]:
df.dtypes

OBJECTID_12                            int64
OBJECTID_1                             int64
OBJECTID                               int64
CITY_ID                                int64
zEstOCI                               object
zInspectOC                           float64
link_id                              float64
mean_traffic_volume                  float64
standard_deviation_traffic_volume    float64
scaled_traffic_volume                 object
dtype: object

In [8]:
df['mean_traffic_volume'].astype(float)

0            NaN
1         9216.0
2         3020.0
3         3235.0
4          115.0
5            NaN
6            NaN
7           29.0
8         2667.0
9           54.5
10          44.0
11         275.5
12          73.0
13          92.0
14         927.5
15         149.0
16         225.0
17        1277.5
18         142.0
19       10086.5
20        9987.0
21          54.0
22         125.0
23        1578.0
24        7773.0
25        1721.5
26          40.0
27           NaN
28         524.0
29         102.5
          ...   
35404      183.5
35405      101.0
35406        NaN
35407        NaN
35408        NaN
35409        NaN
35410       85.5
35411        NaN
35412       42.0
35413        NaN
35414     1367.0
35415        NaN
35416      552.0
35417        NaN
35418       34.0
35419      121.5
35420      830.0
35421     2577.5
35422       62.0
35423      580.5
35424        NaN
35425       45.0
35426        NaN
35427     2824.0
35428       22.0
35429        NaN
35430        NaN
35431      105

In [9]:
df['mean_traffic_volume'] = df.mean_traffic_volume.fillna(-1)

In [10]:
#Need to strip out anything that isn't -1
#So split them into two datasets for recombining later
dftemp1 = df[df['mean_traffic_volume'] > -1].copy()
dftemp2 = df[df['mean_traffic_volume'] == -1].copy()

In [11]:
#Now scale dftemp1
dftemp1['scaled_traffic_volume'] = minmax_scale(dftemp1.mean_traffic_volume, feature_range=(0, 100))

In [12]:
#Let's create an index column that is the following:
# Traffic X  (1-OCI)
dftemp1['EmphasisIndex'] = dftemp1['scaled_traffic_volume'].astype(float)*(100-dftemp1['zInspectOC'])

In [13]:
#Add new column before concatenatin to dftemp2
dftemp2['EmphasisIndex'] = -1

In [14]:
#recombine both temp tables into df
df = pd.concat([dftemp1, dftemp2])

In [15]:
#write to csv
#-1 basically indicates we have incomplete data on it

In [16]:
df.to_csv('scaled_traffic_oci_data.csv')

In [17]:
df.columns

Index(['OBJECTID_12', 'OBJECTID_1', 'OBJECTID', 'CITY_ID', 'zEstOCI',
       'zInspectOC', 'link_id', 'mean_traffic_volume',
       'standard_deviation_traffic_volume', 'scaled_traffic_volume',
       'EmphasisIndex'],
      dtype='object')

In [18]:
#And let's create a version for linking that is much simpler
dfsimplified = df[['CITY_ID', 'EmphasisIndex']]

In [19]:
dfsimplified.to_csv('simple_scaled_traffic_oci_data.csv')

In [21]:
df.columns

Index(['OBJECTID_12', 'OBJECTID_1', 'OBJECTID', 'CITY_ID', 'zEstOCI',
       'zInspectOC', 'link_id', 'mean_traffic_volume',
       'standard_deviation_traffic_volume', 'scaled_traffic_volume',
       'EmphasisIndex'],
      dtype='object')

In [23]:
#Let's also export a random selection of City_IDs to check manually for proper testing
randomsample = df[['CITY_ID', 'mean_traffic_volume', 'zEstOCI']]

In [25]:
#With a population size of 35434 and a Confidence Interval of 95%, 5% margin of error,
#and a 50% population proportion we need a sample of 381
randomsample.shape

(35434, 3)

In [30]:
#Make a random selection
randomsample = randomsample.sample(n=381)
print("random sample size (rows, cols)", randomsample.shape)

random sample size (rows, cols) (381, 3)


In [31]:
#randomsample.to_csv('randomsample.csv')